# Проект для «Викишоп»

**Описание проекта**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель проекта**

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

`/datasets/toxic_comments.csv`
- 'text' - текст комментария;
- 'toxic' — целевой признак.

In [1]:
# импорт библиотек

import pandas as pd

import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.utils import shuffle
from sklearn.utils import resample

from sklearn.metrics import f1_score

from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import RandomUnderSampler

import re
import nltk
from nltk.stem import WordNetLemmatizer 
# from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 

In [2]:
# отключу предупреждения
import warnings
warnings.filterwarnings('ignore')

In [3]:
# кросс-валидация
CV=3
# фиксация результата, получено генератором случайных чисел
STATE=6510571

## Подготовка

### Обзор данных

In [4]:
try:
    data = pd.read_csv('datasets/toxic_comments.csv')
except FileNotFoundError as e:
    print(repr(e))
    data = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
# получаю общую сводку
data.info()

# проверю на корректный вывод
display(data.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
16150,).<<<. I have never done that. I have never ed...,0
42724,"""\nI agree with Jaakobu. I replaced the lead b...",0
87163,editors getting blocked after pointing out wha...,0
63797,WikiTheClown\nWelcome to Wikipedia. Although e...,1
158067,You have mail \n\n Adler,0


In [6]:
# проверю на явные дубликаты
print(f'Количество явных дубликатов: {data.duplicated().sum()}')

Количество явных дубликатов: 0


In [7]:
print('Количество классов в выборке: \n')
print(data.toxic.value_counts())

print('\n ////////////////////////// \n')

print('В процентном соотношении:')
data.toxic.value_counts().div(
    data.toxic.value_counts().sum()
).mul(100).round(2)

Количество классов в выборке: 

0    143346
1     16225
Name: toxic, dtype: int64

 ////////////////////////// 

В процентном соотношении:


0    89.83
1    10.17
Name: toxic, dtype: float64

Предварительные замечания:
- значительный дисбаланс классов, соотношение близкое к 9 к 1.

### Подготовка данных

In [8]:
# подготовка
wnl = WordNetLemmatizer() 

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dsmig\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Лемматизация и очистка текста

In [9]:
def prep_text(text):
    lemm_text =  " ".join([wnl.lemmatize(w)
                           for w in nltk.word_tokenize(text.lower())]) #"".join(wnl.lemmatize(text.lower()))
    clear_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text)
    return ' '.join(clear_text.split())

In [10]:
%%time

data['lemm'] = data['text'].apply(prep_text)

data.sample(5)

CPU times: total: 1min 16s
Wall time: 1min 16s


,text,toxic,lemm
4353,""" HJ Mitchell has no justification for claimin...",0,hj mitchell ha no justification for claiming h...
14489,"""\n\n Softcore porn comedy? \n\nI have replace...",0,softcore porn comedy i have replaced period so...
87318,"""\n\n about asessment of Remo Fernandes \n\nHe...",0,about asessment of remo fernandes hey please a...
156500,"Dear Lord, take this away from him before he d...",0,dear lord take this away from him before he do...
135225,"I have now reinstated that sentence, in a bala...",0,i have now reinstated that sentence in a balan...


Текст очищен от знаков и лемматизирован.

### Разбиение на выборки

Вместо выделения валидационной выборки использую трехкратную кросс-валидацию. Учытвая объем данных, достатчно выделить 10% данных в качестве тестовой выборки.

In [11]:
target = data['toxic']
features = data['lemm']

In [12]:
raw_features_train, raw_features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.10,
                                                                            random_state=STATE)

In [13]:
features_train = count_tf_idf.fit_transform(raw_features_train)
features_test = count_tf_idf.transform(raw_features_test)

### Борьба с дисбалансом классов

Cравнен. метрики полученные при использовании модели Логистической Регрессии на исходных данных, данных с уменьшенным негативным классом, и данных с измененным весом классов.

Меры качества на исходных уменьшенных данных:

In [14]:
pipe_logreg = Pipeline([('tfidf', TfidfVectorizer()), ('model', LogisticRegression(random_state=STATE))])

model_logreg = pipe_logreg.fit(raw_features_train, target_train)

In [15]:
f1_logred_raw = cross_val_score(model_logreg, raw_features_train, target_train, cv=CV, scoring='f1')

print('Значение F1 для исходных данных:', f1_logred_raw.mean())

Значение F1 для исходных данных: 0.7346720776827412


#### Ресэмплинг

In [19]:
imba_pipeline_logreg = make_pipeline(TfidfVectorizer(), RandomUnderSampler(random_state=STATE, sampling_strategy='majority'), LogisticRegression(random_state=STATE))

imba_model_logreg = imba_pipeline_logreg.fit(raw_features_train, target_train)

In [20]:
# проверю результат на исходных данных

f1_logred_resampled = cross_val_score(imba_model_logreg, raw_features_train, target_train, cv=CV, scoring='f1')

print('Значение F1 для данных после ресэмплинга:', f1_logred_resampled.mean())

Значение F1 для данных после ресэмплинга: 0.6765413482335925


#### Изменение веса классов

In [21]:
pipe_logreg = Pipeline([('tfidf', TfidfVectorizer()), ('model', LogisticRegression(random_state=STATE, class_weight='balanced'))])

model_logreg = pipe_logreg.fit(raw_features_train, target_train)

In [22]:
f1_logreg_weight = cross_val_score(model_logreg, raw_features_train, target_train, cv=CV, scoring='f1')

print('Значение F1 для данных на измененном весе классов:', f1_logreg_weight.mean())

Значение F1 для данных на измененном весе классов: 0.7476038783298659


### Вывод

На этапе подготовки был проведен обзор данных в результате которого выявлен значительный дисбаланс классов.  
Данные были подготовлены для использования с алгоритмами машинного обучения, включая лемматизацию текста и очистку от лишних символов, а также использования метода TfidfVectorizer для векторизации текста.  
После было проведено сравнение методов борьбы с дисбалансом классов. В результате сравнений наилучший результат по метрике F1 показало изменение веса классов, тогда как ресэмплинг показал ухудшение значения метрики.

Продолжу работу с исходными данными используя параметр 'balanced' для определения веса классов, остальные выгружу за ненадобностью.

In [23]:
del data, wnl, stopwords, count_tf_idf

## Обучение

Обучу и сравню результаты следующих моделей:
- DecisionTreeClassifier; 
- RandomForestClassifier;
- LogisticRegression;
- LGBMClassifier.

Использую GridSearchCV для подбора гиперпараметров, кроме модели LGBMClassifier для которой использую гиперпараметры по умолчанию.  
Сравню модели по метрике F1 в результате трехкратной кросс-валидации.  
Отберу две модели с наилучшими показателями метрики, переобучу их с найденными гиперпараметрами и сравню их на тестовой выборке без кросс-валидации.  
Проверю лучшую модель на адекватность сравнением с dummy моделью.

### Сравнение моделей

#### DecisionTreeClassifier

In [24]:
%%time

pipe_dtc = Pipeline([('tfidf', TfidfVectorizer()), ('model', DecisionTreeClassifier())])

params_dtc = [{
    'model__max_depth': range(2, 53, 10),
    'model__class_weight':['balanced'],
    'model__random_state':[STATE]
}]

model_dtc = GridSearchCV(pipe_dtc,
                      param_grid=params_dtc,
                      scoring='f1',
                      cv=CV)

model_dtc.fit(raw_features_train, target_train)

CPU times: total: 6min 58s
Wall time: 6min 58s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('model', DecisionTreeClassifier())]),
             param_grid=[{'model__class_weight': ['balanced'],
                          'model__max_depth': range(2, 53, 10),
                          'model__random_state': [6510571]}],
             scoring='f1')

In [25]:
print('Лучшее значение F1 для DecisionTreeClassifier:', model_dtc.best_score_)
'Оптимальные параметры модели:', model_dtc.best_params_

Лучшее значение F1 для DecisionTreeClassifier: 0.5599083736859756


('Оптимальные параметры модели:',
 {'model__class_weight': 'balanced',
  'model__max_depth': 52,
  'model__random_state': 6510571})

#### RandomForestClassifier

In [26]:
%%time

pipe_rfc = Pipeline([('tfidf', TfidfVectorizer()), ('model', RandomForestClassifier())])

params_rfc = [{
    'model__max_depth': range(4, 25, 5),
    'model__n_estimators': range(50, 151, 50),
    'model__class_weight':['balanced'],
    'model__random_state':[STATE]
}]

model_rfc = GridSearchCV(pipe_rfc,
                      param_grid=params_rfc,
                      scoring='f1',
                      cv=CV)

model_rfc.fit(raw_features_train, target_train)

CPU times: total: 7min 15s
Wall time: 7min 15s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('model', RandomForestClassifier())]),
             param_grid=[{'model__class_weight': ['balanced'],
                          'model__max_depth': range(4, 25, 5),
                          'model__n_estimators': range(50, 151, 50),
                          'model__random_state': [6510571]}],
             scoring='f1')

In [27]:
print('Лучшее значение F1 для RandomForestClassifier:', model_rfc.best_score_)
'Оптимальные параметры модели:', model_rfc.best_params_

Лучшее значение F1 для RandomForestClassifier: 0.3974034293158115


('Оптимальные параметры модели:',
 {'model__class_weight': 'balanced',
  'model__max_depth': 24,
  'model__n_estimators': 100,
  'model__random_state': 6510571})

#### LogisticRegression

In [28]:
%%time

pipe_logreg = Pipeline([('tfidf', TfidfVectorizer()), ('model', LogisticRegression())])

params_logreg = [{
    'model__solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'model__max_iter': range(50, 151, 25),
    'model__class_weight':['balanced'],
    'model__random_state':[STATE]
}]

model_logreg = GridSearchCV(pipe_logreg,
                      param_grid=params_logreg,
                      scoring='f1',
                      cv=CV)

model_logreg.fit(raw_features_train, target_train)

CPU times: total: 4min 42s
Wall time: 4min 43s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('model', LogisticRegression())]),
             param_grid=[{'model__class_weight': ['balanced'],
                          'model__max_iter': range(50, 151, 25),
                          'model__random_state': [6510571],
                          'model__solver': ['newton-cg', 'lbfgs',
                                            'liblinear']}],
             scoring='f1')

In [29]:
print('Лучшее значение F1 для LogisticRegression:', model_logreg.best_score_)
'Оптимальные параметры модели:', model_logreg.best_params_

Лучшее значение F1 для LogisticRegression: 0.7478546615507282


('Оптимальные параметры модели:',
 {'model__class_weight': 'balanced',
  'model__max_iter': 75,
  'model__random_state': 6510571,
  'model__solver': 'lbfgs'})

#### LGBMClassifier

In [30]:
%%time

pipe_lgbm = Pipeline([('tfidf', TfidfVectorizer()), ('model', LGBMClassifier(random_state=STATE, class_weight='balanced'))])

model_lgbm = pipe_lgbm.fit(raw_features_train, target_train)

CPU times: total: 3min 46s
Wall time: 21 s


In [31]:
%%time

f1_LGBM = cross_val_score(model_lgbm, raw_features_train, target_train, cv=CV, scoring='f1')

print(f1_LGBM)
print('Значение F1 для LGBMClassifier:', f1_LGBM.mean())

[0.71778656 0.7051983  0.70975484]
Значение F1 для LGBMClassifier: 0.7109132344842605
CPU times: total: 7min 25s
Wall time: 42.6 s


### Тестирование моделей

В результате сравнения моделей наилучший результат показала модель LogisticRegression с F1 = 0.75 (с округлением до сотых в большую сторону).  
Остальные модели не показали результат соотствтетсвующий заданной границе метрики качества. Из них модель LGBMClassifier на втором месте, показала резуьтат F1 = 0.71 и будет использована для проверки на тестовой выборке и сравнения с LogisticRegression.

#### LogisticRegression 

In [32]:
%%time

model_logreg = LogisticRegression(random_state=STATE, solver='liblinear', max_iter=50, class_weight='balanced')

model_logreg.fit(features_train, target_train)

CPU times: total: 1.61 s
Wall time: 1.25 s


LogisticRegression(class_weight='balanced', max_iter=50, random_state=6510571,
                   solver='liblinear')

In [33]:
%%time

logreg_predict = model_logreg.predict(features_test)

CPU times: total: 0 ns
Wall time: 2 ms


In [34]:
logreg_f1 = f1_score(target_test, logreg_predict)
logreg_f1

0.7507970244420828

#### LGBMClassifier

Так как модель LGBMClassifier обучалась с параметрами по умолчанию без использования GridSearchCV - переобучение не требуется.

In [35]:
%%time

model_lgbm = LGBMClassifier(random_state=STATE, class_weight='balanced')

model_lgbm.fit(features_train, target_train)

CPU times: total: 2min 58s
Wall time: 12.2 s


LGBMClassifier(class_weight='balanced', random_state=6510571)

In [36]:
%%time

lgbm_predict = model_lgbm.predict(features_test)

CPU times: total: 891 ms
Wall time: 63.4 ms


In [37]:
lgbm_f1 = f1_score(target_test, lgbm_predict)
lgbm_f1

0.7428412566027246

#### DummyClassifier

Проверка на адекватность.

In [38]:
best_iter = 0
best_f1 = 0
best_strat = []

strat = ['most_frequent', 'prior', 'stratified', 'uniform']

for i in range(len(strat)):
     model_dummy = DummyClassifier(random_state=STATE, strategy=strat[i])
     
     model_dummy.fit(features_train, target_train)
     dummy_predict = model_dummy.predict(features_test)
     
     f1 = f1_score(target_test, dummy_predict)
     if f1 > best_f1:
         best_f1 = f1
         best_strat = strat[i]
 
print(f'Максимальное достигнутое значение f1-меры: {best_f1}, при стратегии: {best_strat}')

Максимальное достигнутое значение f1-меры: 0.17262846892476522, при стратегии: uniform


### Вывод

В результате анализа были отобраны модели LogisticRegression и LGBMClassifier.  
Модель LogisticRegression была переобучена с оптимальными подобранными гиперпараметрами. Модель LGBMClassifier изначально была обучена с гиперпараметрами по умолчанию кроме предуказанных random_state=STATE и class_weight='balanced' и была переобучена с этими параметрами.
Данные модели были протестированы путем предсказания на тестовой выборке и сравнения метрики F1.

В результате тестирования получены следующие результаты метрики F1:
- LogisticRegression - 0.751;
- LGBMClassifier - 0.743.

Модель LogisticRegression показала лучший результат.

## Общий вывод

Заказчиком было дано задание на обучение модели способной классифицировать комментарии на позитивные и негативные. Для этого был предоставлен набор данных с разметкой о токсичности данных.  
В качестве единственной необходимой метрики качества модели была дана метрика F1 c требуемыми показателями не ниже 0.75.

В результате выполнение проекта отобрана модель LogisticRegression c результатом метрики F1 равным 0.751.